In [16]:
# This code block stores file paths to variables to make the code neat
# The stored variables are called in the read_excel function and stored as dataframes

import pandas as pd
import numpy as np

# Store excel file locations to variables (change it as per your path to file)
file_2012=r"C:\Users\warra\Desktop\Freelance\data\data\FemaleMerge\1. Merging by Parts\12. Section 3 Part 1 Consumption Expenditure of the Household on Food Items\2012_s3p1_f.csv"
file_2013=r"C:\Users\warra\Desktop\Freelance\data\data\FemaleMerge\1. Merging by Parts\12. Section 3 Part 1 Consumption Expenditure of the Household on Food Items\2013_s3p1_f.csv"
file_2014=r"C:\Users\warra\Desktop\Freelance\data\data\FemaleMerge\1. Merging by Parts\12. Section 3 Part 1 Consumption Expenditure of the Household on Food Items\2014_s3p1_f.csv"

# Read excel files
df_2012 = pd.read_csv(file_2012)
df_2013 = pd.read_csv(file_2013)
df_2014 = pd.read_csv(file_2014)


In [17]:
'''
This code block will be used to standardize column names across the years to avoid discrepancies during the merging process.
For example in the roaster data for 2013 rq21 and rq23 are not the same as rq21 and rq23 in 2014 data, but they have the same variable names. Hence, we decide to rename such columns beforehand
We will add the updated name to the mapping dictionaries instead of the original names.

'''

# Rename columns in df_2012
df_2012.rename(columns={
    'Fooditems': 's3p1_sr',
    'S3P1Q1': 's3p1_q1',
    'S3P1Q2_UNIT': 's3p1_q2',
    'S3P1Q4': 's3p1_q4',
    'S3P1Q5_UNIT': 's3p1_q5',
    'S3P1Q6_UNIT': 's3p1_q7',
    'S3P1Q7_UNIT': 's3p1_q9',
    'S3P1Q2_CODE': 'CES_HHCPI_Code2w',
    'S3P1Q3': 'CES_HHCPI_PPU',
    'S3P1Q5_CODE': 'CES_CIC_Code2w',
    'S3P1Q6_CODE': 'CES_ProH_Code2w',
    'S3P1Q7_CODE': 'CES_GIC_Code2w',
    'PROVINCE_ID': 'P_ID',
    'DISTRICT_ID': 'D_ID',
    'TEHSIL_ID': 'T_ID',
    'UC_ID': 'UC_ID',
    'MAUZA_ID': 'M_ID'
}, inplace=True)


# df_2014 doesn't need renaming as it is the reference

In [18]:
# Updated mappings

mapping_2012 = [
    'hid', 'round', 's3p1_sr', None, None, 's3p1_q1', 's3p1_q2', None,
    's3p1_q4', 's3p1_q5', None, 's3p1_q7', None, 's3p1_q9', None, None,
    'CES_HHCPI_Code2w', 'CES_HHCPI_PPU', 'CES_CIC_Code2w', 'CES_ProH_Code2w',
    'CES_GIC_Code2w', 'P_ID', 'D_ID', 'T_ID', 'UC_ID', 'M_ID'
]

mapping_2013 = [
     'hid', 'round', 's3p1_sr', 's3p1_item', None, 's3p1_q1', 's3p1_q2', 's3p1_q3', 's3p1_q4',
    's3p1_q5', 's3p1_q6', 's3p1_q7', 's3p1_q8', 's3p1_q9', 's3p1_q10', 's3p1_q11',
    None, None, None, None, None, None, None, None, None, None
]

mapping_2014 = [
     'hid', 'round', 's3p1_sr', 's3p1_item', 's3p1_unit', 's3p1_q1', 's3p1_q2', 's3p1_q3', 's3p1_q4',
    's3p1_q5', 's3p1_q6', 's3p1_q7', 's3p1_q8', 's3p1_q9', 's3p1_q10', 's3p1_q11',
    None, None, None, None, None, None, None, None, None, None
]


In [19]:
# Create a list of all possible columns in the correct order
all_columns = []

for col in mapping_2012:
    if col and col not in all_columns:
        all_columns.append(col)   
for col in mapping_2013:
    if col and col not in all_columns:
        all_columns.append(col)
for col in mapping_2014:
    if col and col not in all_columns:
        all_columns.append(col)



In [20]:
def standardize_and_merge(dfs, mappings, all_columns):
    merged_data = {col: [] for col in all_columns}

    for df, mapping in zip(dfs, mappings):
        print(f"Processing DataFrame with columns: {df.columns.tolist()}")
        for i, col in enumerate(mapping):
            if col:
                ref_col = col.strip()  # Remove leading/trailing whitespace
                if ref_col not in merged_data:
                    merged_data[ref_col] = []
                if ref_col in df.columns:
                    print(f"Appending data for column {ref_col}")
                    if isinstance(df[ref_col], pd.Series):
                        merged_data[ref_col].extend(df[ref_col].tolist())
                    elif isinstance(df[ref_col], pd.DataFrame):
                        print(f"Column {ref_col} is duplicated in DataFrame. Appending data for each duplicate.")
                        for _, series in df[ref_col].items():
                            merged_data[ref_col].extend(series.tolist())
                else:
                    print(f"Column {ref_col} not found in DataFrame. Adding NaNs.")
                    merged_data[ref_col].extend([np.nan] * len(df))
    
    max_len = max(len(v) for v in merged_data.values())
    for key in merged_data:
        col_len = len(merged_data[key])
        if col_len < max_len:
            merged_data[key].extend([np.nan] * (max_len - col_len))

    merged_df = pd.DataFrame.from_dict(merged_data)
    return merged_df


In [21]:
# Usage with dataframes and mappings
dfs = [df_2012, df_2013, df_2014]
mappings = [mapping_2012, mapping_2013, mapping_2014]

merged_df = standardize_and_merge(dfs, mappings, all_columns)

Processing DataFrame with columns: ['Unnamed: 0.1', 'Unnamed: 0', 'hid', 'round', 'hhsize', 's3p1_sr', 's3p1_q1', 's3p1_q2', 'CES_HHCPI_Code2w', 'CES_HHCPI_PPU', 's3p1_q4', 's3p1_q5', 'CES_CIC_Code2w', 's3p1_q7', 'CES_ProH_Code2w', 's3p1_q9', 'CES_GIC_Code2w', 'P_ID', 'D_ID', 'T_ID', 'UC_ID', 'M_ID']
Appending data for column hid
Appending data for column round
Appending data for column s3p1_sr
Appending data for column s3p1_q1
Appending data for column s3p1_q2
Appending data for column s3p1_q4
Appending data for column s3p1_q5
Appending data for column s3p1_q7
Appending data for column s3p1_q9
Appending data for column CES_HHCPI_Code2w
Appending data for column CES_HHCPI_PPU
Appending data for column CES_CIC_Code2w
Appending data for column CES_ProH_Code2w
Appending data for column CES_GIC_Code2w
Appending data for column P_ID
Appending data for column D_ID
Appending data for column T_ID
Appending data for column UC_ID
Appending data for column M_ID
Processing DataFrame with columns: 

In [23]:
# Rename columns for the merged file (if needed)
rename_mapping = {
   'hid': 'HID',
    'round': 'Survey_Round',
    's3p1_sr': 'CES_FoodID',
    's3p1_item': 'CES_FoodName',
    's3p1_unit': 'CES_FoodUnit',
    's3p1_q1': 'CES_HHCI_2w',
    's3p1_q2': 'CES_HHCPI_Qty2w',
    's3p1_q3': 'CES_HHCPI_Val2w',
    's3p1_q4': 'CES_CI',
    's3p1_q5': 'CES_CIC_Qty2w',
    's3p1_q6': 'CES_CIC_Val2w',
    's3p1_q7': 'CES_ProH_Qty2w',
    's3p1_q8': 'CES_ProH_Val2w',
    's3p1_q9': 'CES_GIC_Qty2w',
    's3p1_q10': 'CES_GIC_Val2w',
    's3p1_q11': 'CES_GI_Src'
}

merged_df.rename(columns=rename_mapping, inplace=True)
'''
# Drop redundant columns
merged_df.drop(merged_df.columns[merged_df.columns.str.contains('Unnamed', case=True)], axis=1, inplace=True)
merged_df.drop(merged_df.columns[merged_df.columns.str.contains(' ', case=False)], axis=1, inplace=True)
'''
# Save the merged dataframe to a CSV file
merged_df.to_csv('1. merged_CnE_HH Food Items.csv', index=False)
